# Chain of Thought
This technique requires the language model to describe its solution in logical steps, and is particularly effective at improving answers to mathematical or reasoning questions. 

First, initialize the LLM object with your provider of choice.

In [1]:
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key="KEY-ABC-DEF",
    temperature = 0,
    model_name='gpt-3.5-turbo'
)

c:\SRC\Non-PoC\rag-learning-services\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


This example shows a chain of thought prompt being used with few-shot, to illicit a similar output from the LLM.

In [9]:
example = """Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls.
Each can has 3 tennis balls. How many tennis balls does he have now?
A: Roger started with 5 balls. 2 cans of 3 tennis balls
each is 6 tennis balls. 5 + 6 = 11. The answer is 11.
"""

question = """
Q: The cafeteria had 23 apples.
If they used 20 to make lunch and bought 6 more, how many tennis balls do they have?
A:"""

llm.predict(example + '\n' + question)

'The cafeteria started with 23 apples. They used 20 for lunch, leaving 3 apples. They then bought 6 more apples, bringing the total to 3 + 6 = 9 apples. The cafeteria now has 9 apples.'

This example further explores "Tree of thought", which combines Chain of Thought with a "Panel of Experts".

In [10]:
# Based off https://github.com/dave1010/tree-of-thought-prompting

situation="""
Bob is in the living room.
He walks to the kitchen, carrying a cup.
He puts a ball in the cup and carries the cup to the bedroom.
He turns the cup upside down, then walks to the garden.
He puts the cup down in the garden, then walks to the garage.
Where is the ball?
"""

tree_of_thought="""
Imagine three different experts are answering this question.
All experts will write down 1 step of their thinking,
then share it with the group.
Then all experts will go on to the next step, etc.
If any expert realises they're wrong at any point then they leave.
The question is...
"""

print(llm.predict(tree_of_thought + '\n' + situation))

Expert 1: The ball is in the cup when Bob puts it in the bedroom.
Expert 2: No, the ball falls out of the cup when Bob turns it upside down in the bedroom.
Expert 3: Actually, the ball is left in the cup when Bob puts it down in the garden.


We can use Chain of Thought with vectors in our database as well:

In [13]:
from rag_app.vector_search import VectorSearch

# Initiate the database connection
vector_database = VectorSearch(host='localhost', port=51972, namespace='USER', username='SuperUser', password='sys')

# Load the cached text embedding model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("avsolatorio/GIST-Embedding-v0", cache_folder='.\\huggingface_cache')

In [14]:
input = "How many manuscripts would the Vatican Library need to digitise over its 4 year digitisation project?"

embedding = model.encode(input)

similar_results = vector_database.search_vector_db_with_embedding(embedding.tolist(), top_k=1)
document = similar_results

prompt = f"""
You are a useful assistant. 
These are examples of how you should answer your user's question by thinking through your steps very carefully: 
{example}

This is their question which you should answer, making sure you explain your steps carefully: 
{input}

These are supporting documents that you should use to form your answer to the user, which you can use in your steps: 
{document}
"""

print(prompt)


You are a useful assistant. 
These are examples of how you should answer your user's question by thinking through your steps very carefully: 
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls.
Each can has 3 tennis balls. How many tennis balls does he have now?
A: Roger started with 5 balls. 2 cans of 3 tennis balls
each is 6 tennis balls. 5 + 6 = 11. The answer is 11.


This is their question which you should answer, making sure you explain your steps carefully: 
How many manuscripts would the Vatican Library need to digitise over its 4 year digitisation project?

These are supporting documents that you should use to form your answer to the user, which you can use in your steps: 
[['The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most signifi

In [19]:
llm.predict(prompt)

'To calculate how many manuscripts the Vatican Library would need to digitize over its 4-year digitization project, we need to consider the information provided in the supporting document.\n\nThe Vatican Library began a 4-year project in March 2014 to digitize its collection of manuscripts. The library has a significant collection of historical texts, including 75,000 codices and 1.1 million printed books.\n\nTo determine the number of manuscripts that would need to be digitized, we need to focus on the 75,000 codices, as these are handwritten manuscripts. Therefore, the Vatican Library would need to digitize 75,000 codices over the course of the 4-year project.\n\nTherefore, the Vatican Library would need to digitize 75,000 manuscripts over its 4-year digitization project.'